<a href="https://colab.research.google.com/github/KovryakRuslan312005/MH-Kovryak-Ruslan-3-16/blob/main/%20%D0%9A%D0%BE%D0%B2%D1%80%D1%8F%D0%BA%20%D0%A0%D1%83%D1%81%D0%BB%D0%B0%D0%BD%20%D0%9F%D0%B0%D0%B2%D0%BB%D0%BE%D0%B2%D0%B8%D1%87%20%D0%9B%D0%B0%D0%B1%D0%B0%20%E2%84%965.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ковряк Руслан Павлович Лаб №5

Крок №1 Встановлюємо необхідні бібліотеки

In [ ]:
!pip install -q kaggle scikit-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Крок №2 Завантажуємо необхідний для роботи датасет

In [ ]:
 from google.colab import files
files.upload()  # Завантаж файл kaggle.json з акаунта Kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d iammustafatz/diabetes-prediction-dataset
!unzip diabetes-prediction-dataset.zip


Saving diabetes_prediction_dataset.csv to diabetes_prediction_dataset.csv
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1734, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open diabetes-prediction-dataset.zip, diabetes-prediction-d

Крок №3 Переглядаємо данні з якими будемо працювати

In [ ]:
df = pd.read_csv('diabetes_prediction_dataset.csv')
print(df.shape)
print(df.info())
print(df.head())

(100000, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB
None
   gender   age  hypertension  heart_disease smoking_history    bmi  \
0  Female  80.0             0              1           never  25.19   
1  Female  54.0             0              0         No Info  27.32   
2    Male  28.0             0         

Крок №4 Пілготавлюємо дані до роботи

In [ ]:
print(df.isnull().sum())  # перевірка на пропущені значення
df = pd.get_dummies(df, drop_first=True)  # перетворення категорій
X = df.drop('diabetes', axis=1)
y = df['diabetes']


gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64


Крок №5 Розділяємо данні на тестувальні та тренувальні

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Крок №6 Будуємо моделі

In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RidgeClassifier': RidgeClassifier(),
    'SGDClassifier': SGDClassifier(),
    'SVC': SVC()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))



=== LogisticRegression ===
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     18292
           1       0.87      0.62      0.72      1708

    accuracy                           0.96     20000
   macro avg       0.92      0.80      0.85     20000
weighted avg       0.96      0.96      0.96     20000

[[18129   163]
 [  654  1054]]

=== RidgeClassifier ===
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     18292
           1       0.99      0.28      0.44      1708

    accuracy                           0.94     20000
   macro avg       0.96      0.64      0.70     20000
weighted avg       0.94      0.94      0.92     20000

[[18287     5]
 [ 1223   485]]

=== SGDClassifier ===
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     18292
           1       0.79      0.67      0.72      1708

    accuracy                           0.96 

Крок №7 Налаштовуємо параметри для HalvingGridSearchCV

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}
grid = HalvingGridSearchCV(LogisticRegression(max_iter=1000), param_grid, scoring='accuracy', factor=2, verbose=1)
grid.fit(X_train, y_train)


n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 10000
max_resources_: 80000
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 8
n_resources: 10000
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 1
n_candidates: 4
n_resources: 20000
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 2
n_candidates: 2
n_resources: 40000
Fitting 5 folds for each of 2 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 1
n_resources: 80000
Fitting 5 folds for each of 1 candidates, totalling 5 fits


HalvingGridSearchCV(estimator=LogisticRegression(max_iter=1000), factor=2,
                    param_grid={'C': [0.01, 0.1, 1, 10],
                                'solver': ['liblinear', 'lbfgs']},
                    scoring='accuracy', verbose=1)

Крок №8 Обираємо найкращу модель

In [ ]:
print("Найкращі параметри:", grid.best_params_)
best_model = grid.best_estimator_


Найкращі параметри: {'C': 0.1, 'solver': 'lbfgs'}


Крок №9 Оцінюємо найкращу модель

In [ ]:
y_best_pred = best_model.predict(X_test)
print(classification_report(y_test, y_best_pred))
print(confusion_matrix(y_test, y_best_pred))


              precision    recall  f1-score   support

           0       0.97      0.99      0.98     18292
           1       0.87      0.62      0.72      1708

    accuracy                           0.96     20000
   macro avg       0.92      0.80      0.85     20000
weighted avg       0.96      0.96      0.96     20000

[[18131   161]
 [  657  1051]]


Крок №10 10 ПРОГНОЗІВ

In [ ]:
import random
random_indices = random.sample(range(len(X_test)), 10)
sample_X = X_test.iloc[random_indices]
sample_y = y_test.iloc[random_indices]
sample_pred = best_model.predict(sample_X)

for i in range(10):
    print(f"Запис {i+1}: Справжній = {sample_y.iloc[i]}, Прогноз = {sample_pred[i]}")


Запис 1: Справжній = 0, Прогноз = 0
Запис 2: Справжній = 0, Прогноз = 0
Запис 3: Справжній = 0, Прогноз = 0
Запис 4: Справжній = 0, Прогноз = 0
Запис 5: Справжній = 0, Прогноз = 0
Запис 6: Справжній = 0, Прогноз = 0
Запис 7: Справжній = 0, Прогноз = 0
Запис 8: Справжній = 0, Прогноз = 0
Запис 9: Справжній = 0, Прогноз = 0
Запис 10: Справжній = 0, Прогноз = 0


Висновок: В цій лабораторній роботі був проведений повний аналіз данних про діабет.Будували моделі,обирали яка буде кращою робили прогнози.
